In [1]:
import requests
from bs4 import BeautifulSoup
import json
from collections import OrderedDict
from datetime import datetime
from dateutil.relativedelta import relativedelta
from __future__ import print_function 
import mysql.connector 
from mysql.connector import errorcode
from timeit import default_timer as timer
import time
import copy
import numpy_financial





# import time
# from datetime import datetime
# from math import trunc

C:\Users\dev46\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# MySQL Functions, Connection, and Prep 

In [2]:
cnx = mysql.connector.connect(user='root', password = 'password')
cursor = cnx.cursor()

In [4]:
DB_NAME = 'SEC'
TABLES = {}

#Creating a Database
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)


In [5]:
# Creating Parent Enity "Company_Info"
TABLES['Company_Info'] = (
    "CREATE TABLE Company_Info ("
    "  `Ticker` varchar(6) NOT NULL,"
    "  `CIK` varchar(11) NOT NULL,"
    "  `sic` varchar(25),"
    "  `sic_Description` varchar(100),"
    "  `ein` varchar(25),"
    "  PRIMARY KEY (`CIK`)"
    ") ENGINE=InnoDB")

table_description = TABLES['Company_Info']
try:
    print("Creating table {}: ".format('Company_Info'), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")

Creating table Company_Info: already exists.


In [6]:
def Creating_a_table(name):
    Primary_key = name+'_num'
    Foreign_ID = name+'_FK'
    TABLES[name] = (
        "CREATE TABLE {} ("
        "  `{}` int(25) NOT NULL AUTO_INCREMENT,"
        "  `CIK` varchar(11) NOT NULL,"
        "  `Ticker` varchar(6),"
        "  `val` DECIMAL(25, 2),"
        "  `end` date,"
        "  `frame` varchar(25) NOT NULL,"
        "  `Num_From_Current_Date` varchar(25),"
        "  `filed` varchar(25),"
        "  `accn` varchar(25),"
        "  `fy` varchar(25),"
        "  `fp` varchar(25),"
        "  `form` varchar(25),"
        "  `label` varchar(100),"
        "  PRIMARY KEY (`{}`,`CIK`), KEY `CIK` (`CIK`),"
        "  CONSTRAINT `{}` FOREIGN KEY (`CIK`)"
        "     REFERENCES `Company_Info` (`CIK`) ON DELETE CASCADE"
        ") ENGINE=InnoDB".format(name,Primary_key, Primary_key,Foreign_ID))

    table_description = TABLES[name]
    try:
        print("Creating table {}: ".format(name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


In [7]:
def importing_Company_data(name):
    cnx = mysql.connector.connect(user='root', password = 'password', database='SEC')
    cursor = cnx.cursor()    
    
    add_value = ("INSERT INTO {} "
                   "(Ticker, CIK, sic, sic_Description, ein) "
                   "VALUES (%(Ticker)s, %(CIK)s, %(sic)s, %(sic_Description)s, %(ein)s)".format(name))
    
    data_value = {
        'Ticker': Ticker,
        'CIK': CIK,
        'sic': sic,
        'sic_Description': sic_Description,
        'ein': ein,
    }
    cursor.execute(add_value, data_value)
    emp_no = cursor.lastrowid
    cnx.commit()
    return

In [8]:
def Original_importing_data(name, data):
    cnx = mysql.connector.connect(user='root', password = 'password', database='SEC')
    cursor = cnx.cursor()    
    
    add_value = ("INSERT INTO {} "
                   "(CIK, Ticker, val, end, frame, filed, accn, fy, fp, form, label) "
                   "VALUES (%(CIK)s, %(Ticker)s, %(val)s, %(end)s, %(frame)s, %(filed)s, %(accn)s, %(fy)s, %(fp)s, %(form)s, %(label)s)".format(name))

    for i in data:
        i_str = str(i)
        if data[i] == "" or data[i] == {}:
            data_value = {
              'CIK': CIK,
              'Ticker': Ticker,
              'val': None,
              'end': None,
              'frame': i_str,
              'filed': None,
              'accn': None,
              'fy': None,
              'fp': None,
              'form': None,
              'label': None,
            }
            cursor.execute(add_value, data_value)
            emp_no = cursor.lastrowid
            cnx.commit()
        else:
            datestamp = datetime.strptime(data[i]['end'], '%Y-%m-%d')
            data_value = {
              'CIK': CIK,
              'Ticker': Ticker,
              'val': data[i]['val'],
              'end': datestamp,
              'frame': data[i]['frame'],
              'filed': data[i]['filed'],
              'accn': data[i]['accn'],
              'fy': data[i]['fy'],
              'fp': data[i]['fp'],
              'form': data[i]['form'],
              'label': data[i]['label'],
            }
            cursor.execute(add_value, data_value)
            emp_no = cursor.lastrowid
            cnx.commit()
    return

In [9]:
def importing_data(name, data):
    cnx = mysql.connector.connect(user='root', password = 'password', database='SEC')
    cursor = cnx.cursor()    
    
    add_value = ("INSERT INTO {} "
                   "(CIK, Ticker, val, end, frame, Num_From_Current_Date, filed, accn, fy, fp, form, label) "
                   "VALUES (%(CIK)s, %(Ticker)s, %(val)s, %(end)s, %(frame)s, %(Num_From_Current_Date)s, %(filed)s, %(accn)s, %(fy)s, %(fp)s, %(form)s, %(label)s)".format(name))

    for i in data:
        i_str = str(i)
        if data[i] == "" or data[i] == {}:
            data_value = {
              'CIK': CIK,
              'Ticker': Ticker,
              'val': None,
              'end': None,
              'frame': i_str,
              'Num_From_Current_Date': None,
              'filed': None,
              'accn': None,
              'fy': None,
              'fp': None,
              'form': None,
              'label': None,
            }
            cursor.execute(add_value, data_value)
            emp_no = cursor.lastrowid
            cnx.commit()
        else:
            datestamp = datetime.strptime(data[i]['end'], '%Y-%m-%d')
            data_value = {
              'CIK': CIK,
              'Ticker': Ticker,
              'val': data[i]['val'],
              'end': datestamp,
              'frame': data[i]['frame'],
              'Num_From_Current_Date': data[i]['Num_From_Current_Date'],
              'filed': data[i]['filed'],
              'accn': data[i]['accn'],
              'fy': data[i]['fy'],
              'fp': data[i]['fp'],
              'form': data[i]['form'],
              'label': data[i]['label'],
            }
            cursor.execute(add_value, data_value)
            emp_no = cursor.lastrowid
            cnx.commit()
    return

In [10]:
def Creating_CalcTable():
    cnx = mysql.connector.connect(user='root', password = 'password', database='SEC')
    cursor = cnx.cursor()
    
    Primary_key = 'CalcTable'+'_num'
    Foreign_ID = 'CalcTable'+'_FK'
    TABLES['CalcTable'] = (
        "CREATE TABLE `CalcTable` ("
        "  `{}` int(25) NOT NULL AUTO_INCREMENT,"
        "  `TableRef` varchar(25) NOT NULL,"
        "  `CIK` varchar(11) NOT NULL,"
        "  `Ticker` varchar(6),"
        "  `val` DECIMAL(25, 2),"
        "  `end` date,"
        "  `frame` varchar(25) NOT NULL,"
        "  `filed` varchar(25),"
        "  `form` varchar(25),"
        "  PRIMARY KEY (`{}`,`CIK`), KEY `CIK` (`CIK`),"
        "  CONSTRAINT `{}` FOREIGN KEY (`CIK`)"
        "     REFERENCES `Company_Info` (`CIK`) ON DELETE CASCADE"
        ") ENGINE=InnoDB".format(Primary_key, Primary_key,Foreign_ID))

    table_description = TABLES['CalcTable']
    try:
        print("Creating table {}: ".format('CalcTable'), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")
Creating_CalcTable()

Creating table CalcTable: already exists.


In [11]:
def importing_data_into_CalcTable(data, name='CalcTable'):
    cnx = mysql.connector.connect(user='root', password = 'password', database='SEC')
    cursor = cnx.cursor()    
    
    add_value = ("INSERT INTO {} "
                   "(TableRef, CIK, Ticker, val, end, frame, filed, form) "
                   "VALUES (%(TableRef)s, %(CIK)s, %(Ticker)s, %(val)s, %(end)s, %(frame)s, %(filed)s, %(form)s)".format(name))

    for i in data:
        i_str = str(i)
        if data[i] == "" or data[i] == {}:
            x=1
#             data_value = {
#               'TableRef':  data[i]['TableRef'],
#               'CIK': CIK,
#               'Ticker': Ticker,
#               'val': None,
#               'end': None,
#               'frame':  data[i]['frame'],
#               'filed': None,
#               'form': None,
#             }
#             cursor.execute(add_value, data_value)
#             emp_no = cursor.lastrowid
#             cnx.commit()
        else:
            datestamp = datetime.strptime(data[i]['end'], '%Y-%m-%d')
            data_value = {
              'TableRef':  data[i]['TableRef'],
              'CIK': CIK,
              'Ticker': Ticker,
              'val': data[i]['val'],
              'end': datestamp,
              'frame': data[i]['frame'],
              'filed': data[i]['filed'],
              'form': data[i]['form'],
            }
            cursor.execute(add_value, data_value)
            emp_no = cursor.lastrowid
            cnx.commit()
    return

# Think or Swim

In [12]:
def ThinkORSwim(Ticker):
    try:
        #Years wanting to go back 
        years = 1
        # datetime object containing current date and time
        now = datetime.now()
        # dd/mm/YY H:M:S
        dt_string = now.strftime("%Y/%m/%d %H:%M:%S")
        p='%Y/%m/%d %H:%M:%S'
        epoch = int(time.mktime(time.strptime(dt_string,p)))
        current_time = epoch*1000
        end_time = current_time - (years * 31556926000)
        # print(current_time)
        # print(end_time)
        # print(now)

        endpoint = f'https://api.tdameritrade.com/v1/marketdata/{Ticker}/pricehistory'
        payload = {'apikey': '',
            'periodType':'month',
            'period': '1',
            'frequencyType': 'daily',
            'frequency': '1',
            # Most recent date
            'endDate': current_time,
            # Date in the past
            'startDate': end_time,
            'needExtendedHoursData': 'false'}
        content = requests.get(url = endpoint, params = payload)
        data = content.json()
        last_close = data['candles'][-1]['close']
    except: 
        last_close = None
    return last_close

# Ticker = 'AAPL'
# ThinkORSwim(Ticker)

# Functions Connecting to Yahoo's Data Analytics to MySQL

In [13]:
cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
cursor = cnx.cursor()

TABLES = {}
TABLES['Yahoo_Forecast'] = (
    "CREATE TABLE `Yahoo_Forecast` ("
    "  `CIK` varchar(11) NOT NULL,"
    "  `Ticker` varchar(6) NOT NULL,"
    "  `nextYR_gr` DECIMAL(6, 2),"
    "  `fiveYR_gr` DECIMAL(6, 2),"
    "  `close` DECIMAL(8, 2), "
    "  `Current_Earnings` DECIMAL(8, 2), "
    "  `Future_Earnings` DECIMAL(8, 2), "
    "  `Rev_gr` DECIMAL(8, 2), "
    "  `Current_PE` DECIMAL(8, 2), "
    "  PRIMARY KEY (`Ticker`,`CIK`), KEY `CIK` (`CIK`),"
    "  CONSTRAINT `Company_Info` FOREIGN KEY (`CIK`)"
    "     REFERENCES `Company_Info` (`CIK`)  ON DELETE CASCADE"
    ") ENGINE=InnoDB")

table_description = TABLES['Yahoo_Forecast']
try:
    print("Creating table Yahoo_Forecast: ", end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")


Creating table Yahoo_Forecast: already exists.


In [14]:
# ticker = 'jnj'

def YAHOO(ticker):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
    
#     Anaylist Tab 

    url = f'https://finance.yahoo.com/quote/{ticker}/analysis?p={ticker}'
    r = requests.get(url, headers=headers)
    doc = BeautifulSoup(r.text, 'html.parser')
    
    # Current P/E, Future P/E
    # /html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/table[1]/tbody/tr[2]/td[4]
    # /html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/table[1]/tbody/tr[2]/td[5]    
    try:
        table = doc.find_all('table')[0]
        tbody = table.find('tbody')
        tr = tbody.find_all('tr')[1]
    except:
        data = None 
    
    try:
        Current_Earnings = float(tr.find_all('td')[3].text)
    except:
        Current_Earnings = None
    try:
        Future_Earnings = float(tr.find_all('td')[4].text)
    except:
        Future_Earnings = None 

    # Revenue 
    # /html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/table[2]/tbody/tr[6]/td[5]
    try:
        table = doc.find_all('table')[1]
        tbody = table.find('tbody')
        tr = tbody.find_all('tr')[5]
    except: 
        data = None
    try: 
        Rev_gr = tr.find_all('td')[4].text
        Rev_gr = Rev_gr.replace('%', '') 
        Rev_gr = float(Rev_gr)
    except:
        Rev_gr = None

    # 5 yr Forcast 
    # /html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/table[6]/tbody/tr[5]/td[2]
    try:
        table = doc.find_all('table')[5]
        tbody = table.find('tbody')
        tr = tbody.find_all('tr')[4]
        fiveYr_growth_rate = tr.find_all('td')[1].text
    except: 
        data = None 
    try:
        fiveYr_growth_rate = float(fiveYr_growth_rate.replace('%',''))
    except:
        fiveYr_growth_rate = None

    # Next yr Forcast
    # /html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/table[6]/tbody/tr[4]/td[2]
    try:
        tr = tbody.find_all('tr')[3]
        nextYR_growth_rate = tr.find_all('td')[1].text
    except:
        data = None
    
    try:
        nextYR_growth_rate = float(nextYR_growth_rate.replace('%',''))
    except:
        nextYR_growth_rate = None
        
    # Current_Price
    try: 
        price = float(doc.find_all('fin-streamer')[15].next_element)
    except:
        try: 
            price = float(doc.find_all('fin-streamer')[15].next_element)
        except:
            price = None
    return(nextYR_growth_rate, fiveYr_growth_rate, price, Current_Earnings, Future_Earnings, Rev_gr)

# YAHOO('HELE')

In [15]:
def importing_Yahoo_Forecast(nextYR_gr, fiveYR_gr, price, Current_Earnings, Future_Earnings, Rev_gr, Current_PE):
    cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
    cursor = cnx.cursor()
    
    add_value = ("INSERT INTO yahoo_forecast "
               "(CIK, Ticker, nextYR_gr, fiveYR_gr, close, Current_Earnings, Future_Earnings, Rev_gr, Current_PE) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")
    
    data_value = (CIK, Ticker, nextYR_gr, fiveYR_gr, price, Current_Earnings, Future_Earnings, Rev_gr, Current_PE)
    
    cursor.execute(add_value, data_value)
    emp_no = cursor.lastrowid
    cnx.commit()
    return


In [16]:
# Creating futureEarningsDate
cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
cursor = cnx.cursor()


name = 'futureEarningsDate'
Primary_key = name+'_num'
Foreign_ID = name+'_FK'
TABLES[name] = (
    "CREATE TABLE {} ("
    "  `{}` int(25) NOT NULL AUTO_INCREMENT,"
    "  `CIK` varchar(11) NOT NULL,"
    "  `Ticker` varchar(11), "
    "  `frame` varchar(11), "
    "  `end` date,"
    "  PRIMARY KEY (`{}`,`CIK`), KEY `CIK` (`CIK`),"
    "  CONSTRAINT `{}` FOREIGN KEY (`CIK`)"
    "     REFERENCES `Company_Info` (`CIK`) ON DELETE CASCADE"
    ") ENGINE=InnoDB".format(name,Primary_key, Primary_key,Foreign_ID))

table_description = TABLES[name]
try:
    print("Creating table {}: ".format(name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")


Creating table futureEarningsDate: already exists.


In [17]:
def importing_data_futureEarningsDate(data):
    
    cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
    cursor = cnx.cursor()
    add_value = ("INSERT INTO futureEarningsDate "
                   "(CIK, Ticker, end, frame) "
                   "VALUES (%(CIK)s, %(Ticker)s, %(end)s, %(frame)s)")

    for i in data:
        i_str = str(i)
        if data[i] == "" or data[i] == {}:
            data_value = {
              'CIK': CIK,
              'Ticker': Ticker,
              'end': None,
              'frame': i_str,
            }
            cursor.execute(add_value, data_value)
            emp_no = cursor.lastrowid
            cnx.commit()
        else:
            data_value = {
              'CIK': CIK,
              'Ticker': Ticker,
              'end': data[i],
              'frame': i_str,
            }
            cursor.execute(add_value, data_value)
            emp_no = cursor.lastrowid
            cnx.commit()
    return

# importing_data_futureEarningsDate(grabbingData)

# SEC Functions

In [18]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
url = f'https://www.sec.gov/files/company_tickers.json'
r = requests.get(url, headers=headers)
CIK_soup = r.json()

def CIK_Finder(Ticker):
    for items in CIK_soup:
        item = CIK_soup[items]
        if item['ticker'] == Ticker:
            CIK = str(item['cik_str'])
            break
    
    Zeros = ''
    if len(CIK) != 10:
        Num_Zeros = 10 - len(CIK)
        for num in range(Num_Zeros):
            Zeros = Zeros + '0'
        CIK = Zeros + CIK

    return CIK


In [19]:
def SEC_link(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
    Scanner = f'{url}'
    r = requests.get(Scanner, headers=headers)
    soup = r.json()
#     soup = BeautifulSoup(r.text,'html.parser')
#     soup = str(soup)
#     soup = json.loads(soup)
    return soup

In [20]:
# x=1
def Split(x):
    Split_List = {}
    try:
        for i in copy.deepcopy(soup['facts']['us-gaap']['StockholdersEquityNoteStockSplitConversionRatio1']['units']['pure']):
            try:
                if i['frame']!='':
                    Split_List[i['frame'][:-1]]= i
            except:
                x=x
    except:
        res = Split_List
    res = dict(reversed(list(Split_List.items())))
    return(res)
# Split(1)

In [21]:
def Shares(USD_List):
    Share_List = {}
    #importing data
    YR_List = {}
    QR_List = {}

    #Creating a dictionary list 
    Oldest_YR = 9999
    for Label in USD_List:
        try: 
            for i in copy.deepcopy(soup['facts']['us-gaap'][Label]['units']['shares']):
                try:
                    if i['frame'] != '':
                        if int(i['frame'][2:6])<Oldest_YR:
                            Oldest_YR = int(i['frame'][2:6])
                        break
                except:
                    x=1 
        except: 
            x=1

    #Format of Dictionary 
    Yrs_difference = Current_Year-Oldest_YR+1
    for num in range(Yrs_difference):
        YR='CY'+ str(num+Oldest_YR)
        Q=YR+'Q'
        try:
            x = QR_List[YR]
        except: 
            QR_List[YR]=''
        for num2 in range(4):
            R = num2+1
            QR = Q + str(R)
            try:
                x = QR_List[QR]
            except: 
                QR_List[QR]=''

    #Importing data
    for Label in USD_List:
        try:
            for i in copy.deepcopy(soup['facts']['us-gaap'][Label]['units']['shares']):
                try:
                    frame = i['frame']
                    frame = frame.replace('I','')
                    try: 
                        if QR_List[frame] == '':
                            QR_List[frame]= i
                            QR_List[frame]['CIK']=CIK
                            QR_List[frame]['Ticker']=Ticker
                            QR_List[frame]['frame']=frame
                            QR_List[frame]['label']=Label
                    except:
                        if YR_List[frame] == '':
                            YR_List[frame]= i
                            YR_List[frame]['CIK']=CIK
                            YR_List[frame]['Ticker']=Ticker
                            YR_List[frame]['frame']=frame
                            YR_List[frame]['label']=Label
                except:
                    x=1
        except: 
            x=1
    Share_List = QR_List 

    #Reversing the Order to most recent data on top
    Share_List = dict(reversed(list(Share_List.items())))

    #Share Conversion 
    for Key in Share_List:
        for i in Key:
            try:
                Current_Val = Share_List[Key][i]['val']
                Share_date = datetime.strptime(Share_List[Key][i]['end'], '%Y-%m-%d')
                for items in Split_Hist:
                    Split_date = datetime.strptime(Split_Hist[items]['end'], '%Y-%m-%d')
                    Split_val = Split_Hist[items]['val']
                    try:
                        if Split_date>Share_date:
                            Difference = abs(Previous_Val-Current_Val)
                            Split_Conversion = (Current_Val/Split_val)
                            Conversion_Diff =  abs(Previous_Val-Split_Conversion)
                            if Conversion_Diff<Difference:
                                Share_List[i]['val'] = round(Split_Conversion,0)
                                Current_Val = Split_Conversion
                            Difference = abs(Previous_Val-Current_Val)
                            Split_Conversion = (Current_Val*Split_val)
                            Conversion_Diff =  abs(Previous_Val-Split_Conversion)
                            if Conversion_Diff<Difference:
                                Share_List[i]['val'] = round(Split_Conversion,0)
                                Current_Val = Split_Conversion
                    except:
                        Previous_Val = Current_Val
                Previous_Val = Current_Val
            except:
                x=1
    return(Share_List)
# Shares(['WeightedAverageNumberOfDilutedSharesOutstanding'])

In [22]:
def USD_gaap(USD_List):
    # def USD_Shares(label)
    YR_List = {}
    QR_List = {}

    #Creating a dictionary list 
    Oldest_YR = 9999
    for Label in USD_List:
        try: 
            for i in copy.deepcopy(soup['facts']['us-gaap'][Label]['units']['USD']):
                try:
                    if i['frame'] != '':
                        if int(i['frame'][2:6])<Oldest_YR:
                            Oldest_YR = int(i['frame'][2:6])
                        break
                except:
                    x=1 
        except: 
            x=1

    #Format of Dictionary 
    Yrs_difference = Current_Year-Oldest_YR+1
    for num in range(Yrs_difference):
        YR='CY'+ str(num+Oldest_YR)
        Q=YR+'Q'
        try:
            x = YR_List[YR]
        except: 
            YR_List[YR]=''
        for num2 in range(4):
            R = num2+1
            QR = Q + str(R)
            try:
                x = QR_List[QR]
            except: 
                QR_List[QR]=''

    #Importing data
    for Label in USD_List:
        try:
            for i in copy.deepcopy(soup['facts']['us-gaap'][Label]['units']['USD']):
                try:
                    frame = i['frame']
                    frame = frame.replace('I','')
                    try: 
                        if QR_List[frame] == '':
                            QR_List[frame]= i
                            QR_List[frame]['CIK']=CIK
                            QR_List[frame]['Ticker']=Ticker
                            QR_List[frame]['frame']=frame
                            QR_List[frame]['label']=Label
                    except:
                        if YR_List[frame] == '':
                            YR_List[frame]= i
                            YR_List[frame]['CIK']=CIK
                            YR_List[frame]['Ticker']=Ticker
                            YR_List[frame]['frame']=frame
                            YR_List[frame]['label']=Label
                except:
                    x=1
        except: 
            x=1

    try:
        # Finding the current frame for Fiscal Year 
        for Key in YR_List:
            try:
                Current_Frame = YR_List[Key]['frame']
            except TypeError:
                x=1

        # Counting the distance till current frame
        counter = 0 
        for Key in YR_List:
            counter += 1
            if Key == Current_Frame:
                Distance_to_current_frame = counter 
                break

        # Filling in Num_From_Current_Date
        counter = 0 
        for Key in YR_List:
            counter += 1
            try:
                Num_From_Current_Date = Distance_to_current_frame-counter
                YR_List[Key]['Num_From_Current_Date'] = Num_From_Current_Date
            except TypeError:
                x=1

        Grouped_data = autofillMisingQuarter(YR_List, QR_List)
        QR_List = Grouped_data[1]
    except: 
        YR_List=YR_List

    # Finding the current frame for Quarterly 
    for Key in QR_List:
        try:
            # This is uses to trigger an error for keys without a frame
            Frame = QR_List[Key]['frame']
            Current_Frame = Key
        except TypeError:
            x=1
            
    # Counting the distance till Quarterly
    counter = 0 
    for Key in QR_List:
        counter += 1
        if Key == Current_Frame:
            Distance_to_current_frame = counter 
            break

    # Filling in Num_From_Current_Date
    counter = 0 
    for Key in QR_List:
        counter += 1
        try:
            Num_From_Current_Date = Distance_to_current_frame-counter
            QR_List[Key]['Num_From_Current_Date'] = Num_From_Current_Date
        except TypeError:
            x=1

    return(YR_List,QR_List)


In [23]:
def USD_gaap_per_shares(USD_List):
    # def USD_Shares(label)
    YR_List = {}
    QR_List = {}

    #Creating a dictionary list 
    Oldest_YR = 9999
    for Label in USD_List:
        try: 
            for i in copy.deepcopy(soup['facts']['us-gaap'][Label]['units']['USD/shares']):
                try:
                    if i['frame'] != '':
                        if int(i['frame'][2:6])<Oldest_YR:
                            Oldest_YR = int(i['frame'][2:6])
                        break
                except:
                    x=1 
        except: 
            x=1
            
    #Format of Dictionary 
    Yrs_difference = Current_Year-Oldest_YR+1
    for num in range(Yrs_difference):
        YR='CY'+ str(num+Oldest_YR)
        Q=YR+'Q'
        try:
            x = YR_List[YR]
        except: 
            YR_List[YR]=''
        for num2 in range(4):
            R = num2+1
            QR = Q + str(R)
            try:
                x = QR_List[QR]
            except: 
                QR_List[QR]=''

    #Importing data
    for Label in USD_List:
        try:
            for i in copy.deepcopy(soup['facts']['us-gaap'][Label]['units']['USD/shares']):
                try:
                    frame = i['frame']
                    frame = frame.replace('I','')
                    try: 
                        if QR_List[frame] == '':
                            QR_List[frame]= i
                            QR_List[frame]['CIK']=CIK
                            QR_List[frame]['Ticker']=Ticker
                            QR_List[frame]['frame']=frame
                            QR_List[frame]['label']=Label
                    except:
                        if YR_List[frame] == '':
                            YR_List[frame]= i
                            YR_List[frame]['CIK']=CIK
                            YR_List[frame]['Ticker']=Ticker
                            YR_List[frame]['frame']=frame
                            YR_List[frame]['label']=Label
                except:
                    x=1
        except: 
            x=1
            
    try:
        # Finding the current frame for Fiscal Year 
        for Key in YR_List:
            try:
                Current_Frame = YR_List[Key]['frame']
            except TypeError:
                x=1

        # Counting the distance till current frame
        counter = 0 
        for Key in YR_List:
            counter += 1
            if Key == Current_Frame:
                Distance_to_current_frame = counter 
                break

        # Filling in Num_From_Current_Date
        counter = 0 
        for Key in YR_List:
            counter += 1
            try:
                Num_From_Current_Date = Distance_to_current_frame-counter
                YR_List[Key]['Num_From_Current_Date'] = Num_From_Current_Date
            except TypeError:
                x=1

        Grouped_data = autofillMisingQuarter(YR_List, QR_List)
        QR_List = Grouped_data[1]
    except: 
        YR_List=YR_List

    # Finding the current frame for Quarterly 
    for Key in QR_List:
        try:
            # This is uses to trigger an error for keys without a frame
            Frame = QR_List[Key]['frame']
            Current_Frame = Key
        except TypeError:
            x=1
            
    # Counting the distance till Quarterly
    counter = 0 
    for Key in QR_List:
        counter += 1
        if Key == Current_Frame:
            Distance_to_current_frame = counter 
            break

    # Filling in Num_From_Current_Date
    counter = 0 
    for Key in QR_List:
        counter += 1
        try:
            Num_From_Current_Date = Distance_to_current_frame-counter
            QR_List[Key]['Num_From_Current_Date'] = Num_From_Current_Date
        except TypeError:
            x=1            
            
    return(YR_List,QR_List)

In [24]:
def Subtraction(Numerator, Denominator):
    Calc = {}
    Search_in_List = []
    if len(Numerator)>=len(Denominator):
        for items in Denominator:
            try:
                if Denominator[items]['end'] not in Search_in_List:
                    Search_in_List.append(Denominator[items]['end'])
            except: 
                x=1
        for i in Numerator:
            for items in Denominator:
                try:
                    Key = str(i)
                    if Numerator[i]['end'] == Denominator[items]['end']:
                        Value = round(Numerator[i]['val']-Denominator[items]['val'],2)
                        Calc[Key] = {}
                        Calc[Key]['CIK'] = CIK
                        Calc[Key]['Ticker'] = Ticker
                        Calc[Key]['val'] = Value
                        Calc[Key]['end'] = Numerator[i]['end']
                        Calc[Key]['frame'] = Numerator[i]['frame']
                        Calc[Key]['filed'] = Numerator[i]['filed']
                        Calc[Key]['accn'] = Numerator[i]['accn']
                        Calc[Key]['fy'] = Numerator[i]['fy']
                        Calc[Key]['fp'] = Numerator[i]['fp']
                        Calc[Key]['form'] = Numerator[i]['form']
                        Calc[Key]['Num_From_Current_Date'] = Numerator[i]['Num_From_Current_Date']
                        Calc[Key]['label'] = ''
                    elif Numerator[i]['end'] not in Search_in_List:
                        try: 
                            x = Calc[Key]
                        except:
                            Calc[Key] = {} 

                except Exception as err:
                    try: 
                        x = Calc[Key]
                    except:
                        Calc[Key] = {} 
    else:
        for i in Numerator:
            try:
                if Numerator[i]['end'] not in Search_in_List:
                    Search_in_List.append(Numerator[i]['end'])
            except: 
                x=1
        for items in Denominator:
            for i in Numerator:
                try:
                    Key = str(i)
                    if Numerator[i]['end'] == Denominator[items]['end']:
                        Value = round(Numerator[i]['val']-Denominator[items]['val'],2)
                        Calc[Key] = {}
                        Calc[Key]['CIK'] = CIK
                        Calc[Key]['Ticker'] = Ticker
                        Calc[Key]['val'] = Value
                        Calc[Key]['end'] = Numerator[i]['end']
                        Calc[Key]['frame'] = Numerator[i]['frame']
                        Calc[Key]['filed'] = Numerator[i]['filed']
                        Calc[Key]['accn'] = Numerator[i]['accn']
                        Calc[Key]['fy'] = Numerator[i]['fy']
                        Calc[Key]['fp'] = Numerator[i]['fp']
                        Calc[Key]['form'] = Numerator[i]['form']
                        Calc[Key]['Num_From_Current_Date'] = Numerator[i]['Num_From_Current_Date']
                        Calc[Key]['label'] = ''
                    elif Denominator[items]['end'] not in Search_in_List:
                        try: 
                            x = Calc[Key]
                        except:
                            Calc[Key] = {} 
                        
                except Exception as err:
                    try: 
                        x = Calc[Key]
                    except:
                        Calc[Key] = {} 
    return(Calc)

In [25]:
def Addition(Numerator, Denominator):
    Calc = {}
    Search_in_List = []
    if len(Numerator)>=len(Denominator):
        for items in Denominator:
            try:
                if Denominator[items]['end'] not in Search_in_List:
                    Search_in_List.append(Denominator[items]['end'])
            except: 
                x=1
        for i in Numerator:
            for items in Denominator:
                try:
                    Key = str(i)
                    if Numerator[i]['end'] == Denominator[items]['end']:
                        Value = round(Numerator[i]['val']+Denominator[items]['val'],2)
                        Calc[Key] = {}
                        Calc[Key]['CIK'] = CIK
                        Calc[Key]['Ticker'] = Ticker
                        Calc[Key]['val'] = Value
                        Calc[Key]['end'] = Numerator[i]['end']
                        Calc[Key]['frame'] = Numerator[i]['frame']
                        Calc[Key]['filed'] = Numerator[i]['filed']
                        Calc[Key]['accn'] = Numerator[i]['accn']
                        Calc[Key]['fy'] = Numerator[i]['fy']
                        Calc[Key]['fp'] = Numerator[i]['fp']
                        Calc[Key]['form'] = Numerator[i]['form']
                        Calc[Key]['Num_From_Current_Date'] = Numerator[i]['Num_From_Current_Date']
                        Calc[Key]['label'] = ''
                    elif Numerator[i]['end'] not in Search_in_List:
                        try: 
                            x = Calc[Key]
                        except:
                            Calc[Key] = {} 

                except Exception as err:
                    try: 
                        x = Calc[Key]
                    except:
                        Calc[Key] = {} 
    else:
        for i in Numerator:
            try:
                if Numerator[i]['end'] not in Search_in_List:
                    Search_in_List.append(Numerator[i]['end'])
            except: 
                x=1
        for items in Denominator:
            for i in Numerator:
                try:
                    Key = str(i)
                    if Numerator[i]['end'] == Denominator[items]['end']:
                        Value = round(Numerator[i]['val']+Denominator[items]['val'],2)
                        Calc[Key] = {}
                        Calc[Key]['CIK'] = CIK
                        Calc[Key]['Ticker'] = Ticker
                        Calc[Key]['val'] = Value
                        Calc[Key]['end'] = Numerator[i]['end']
                        Calc[Key]['frame'] = Numerator[i]['frame']
                        Calc[Key]['filed'] = Numerator[i]['filed']
                        Calc[Key]['accn'] = Numerator[i]['accn']
                        Calc[Key]['fy'] = Numerator[i]['fy']
                        Calc[Key]['fp'] = Numerator[i]['fp']
                        Calc[Key]['form'] = Numerator[i]['form']
                        Calc[Key]['Num_From_Current_Date'] = Numerator[i]['Num_From_Current_Date']
                        Calc[Key]['label'] = ''
                    elif Denominator[items]['end'] not in Search_in_List:
                        try: 
                            x = Calc[Key]
                        except:
                            Calc[Key] = {} 
                        
                except Exception as err:
                    try: 
                        x = Calc[Key]
                    except:
                        Calc[Key] = {} 
    return(Calc)

In [26]:
def Division(Numerator, Denominator):
    Calc = {}
    Search_in_List = []
    if len(Numerator)>=len(Denominator):
        for items in Denominator:
            try:
                if Denominator[items]['end'] not in Search_in_List:
                    Search_in_List.append(Denominator[items]['end'])
            except: 
                x=1
        for i in Numerator:
            for items in Denominator:
                try:
                    Key = str(i)
                    if Numerator[i]['end'] == Denominator[items]['end']:
                        Value = round(Numerator[i]['val']/Denominator[items]['val'],2)
                        Calc[Key] = {}
                        Calc[Key]['CIK'] = CIK
                        Calc[Key]['Ticker'] = Ticker
                        Calc[Key]['val'] = Value
                        Calc[Key]['end'] = Numerator[i]['end']
                        Calc[Key]['frame'] = Numerator[i]['frame']
                        Calc[Key]['filed'] = Numerator[i]['filed']
                        Calc[Key]['accn'] = Numerator[i]['accn']
                        Calc[Key]['fy'] = Numerator[i]['fy']
                        Calc[Key]['fp'] = Numerator[i]['fp']
                        Calc[Key]['form'] = Numerator[i]['form']
                        Calc[Key]['Num_From_Current_Date'] = Numerator[i]['Num_From_Current_Date']
                        Calc[Key]['label'] = ''
                    elif Numerator[i]['end'] not in Search_in_List:
                        try: 
                            x = Calc[Key]
                        except:
                            Calc[Key] = {} 

                except Exception as err:
                    try: 
                        x = Calc[Key]
                    except:
                        Calc[Key] = {} 
    else:
        for i in Numerator:
            try:
                if Numerator[i]['end'] not in Search_in_List:
                    Search_in_List.append(Numerator[i]['end'])
            except: 
                x=1
        for items in Denominator:
            for i in Numerator:
                try:
                    Key = str(i)
                    if Numerator[i]['end'] == Denominator[items]['end']:
                        Value = round(Numerator[i]['val']/Denominator[items]['val'],2)
                        Calc[Key] = {}
                        Calc[Key]['CIK'] = CIK
                        Calc[Key]['Ticker'] = Ticker
                        Calc[Key]['val'] = Value
                        Calc[Key]['end'] = Numerator[i]['end']
                        Calc[Key]['frame'] = Numerator[i]['frame']
                        Calc[Key]['filed'] = Numerator[i]['filed']
                        Calc[Key]['accn'] = Numerator[i]['accn']
                        Calc[Key]['fy'] = Numerator[i]['fy']
                        Calc[Key]['fp'] = Numerator[i]['fp']
                        Calc[Key]['form'] = Numerator[i]['form']
                        Calc[Key]['Num_From_Current_Date'] = Numerator[i]['Num_From_Current_Date']
                        Calc[Key]['label'] = ''
                    elif Denominator[items]['end'] not in Search_in_List:
                        try: 
                            x = Calc[Key]
                        except:
                            Calc[Key] = {} 
                        
                except Exception as err:
                    try: 
                        x = Calc[Key]
                    except:
                        Calc[Key] = {} 
    return(Calc)

In [27]:
def Division_Flipped(Denominator, Numerator,):
    Calc = {}
    Search_in_List = []
    if len(Numerator)>=len(Denominator):
        for items in Denominator:
            try:
                if Denominator[items]['end'] not in Search_in_List:
                    Search_in_List.append(Denominator[items]['end'])
            except: 
                x=1
        for i in Numerator:
            for items in Denominator:
                try:
                    Key = str(i)
                    if Numerator[i]['end'] == Denominator[items]['end']:
                        Value = round(Denominator[items]['val']/Numerator[i]['val'],2)
                        Calc[Key] = {}
                        Calc[Key]['CIK'] = CIK
                        Calc[Key]['Ticker'] = Ticker
                        Calc[Key]['val'] = Value
                        Calc[Key]['end'] = Numerator[i]['end']
                        Calc[Key]['frame'] = Numerator[i]['frame']
                        Calc[Key]['filed'] = Numerator[i]['filed']
                        Calc[Key]['accn'] = Numerator[i]['accn']
                        Calc[Key]['fy'] = Numerator[i]['fy']
                        Calc[Key]['fp'] = Numerator[i]['fp']
                        Calc[Key]['form'] = Numerator[i]['form']
                        Calc[Key]['Num_From_Current_Date'] = Numerator[i]['Num_From_Current_Date']
                        Calc[Key]['label'] = ''
                    elif Numerator[i]['end'] not in Search_in_List:
                        try: 
                            x = Calc[Key]
                        except:
                            Calc[Key] = {} 

                except Exception as err:
                    try: 
                        x = Calc[Key]
                    except:
                        Calc[Key] = {} 
    else:
        for i in Numerator:
            try:
                if Numerator[i]['end'] not in Search_in_List:
                    Search_in_List.append(Numerator[i]['end'])
            except: 
                x=1
        for items in Denominator:
            for i in Numerator:
                try:
                    Key = str(i)
                    if Numerator[i]['end'] == Denominator[items]['end']:
                        Value = round(Denominator[items]['val']/Numerator[i]['val'],2)
                        Calc[Key] = {}
                        Calc[Key]['CIK'] = CIK
                        Calc[Key]['Ticker'] = Ticker
                        Calc[Key]['val'] = Value
                        Calc[Key]['end'] = Numerator[i]['end']
                        Calc[Key]['frame'] = Numerator[i]['frame']
                        Calc[Key]['filed'] = Numerator[i]['filed']
                        Calc[Key]['accn'] = Numerator[i]['accn']
                        Calc[Key]['fy'] = Numerator[i]['fy']
                        Calc[Key]['fp'] = Numerator[i]['fp']
                        Calc[Key]['form'] = Numerator[i]['form']
                        Calc[Key]['Num_From_Current_Date'] = Numerator[i]['Num_From_Current_Date']
                        Calc[Key]['label'] = ''
                    elif Denominator[items]['end'] not in Search_in_List:
                        try: 
                            x = Calc[Key]
                        except:
                            Calc[Key] = {} 
                        
                except Exception as err:
                    try: 
                        x = Calc[Key]
                    except:
                        Calc[Key] = {} 
    return(Calc)

In [28]:
def Append(Label, Calc):
    if Label == {}:
        New_Dictionary = Calc
    elif Calc == {}:
        New_Dictionary = Label
    else:
        Oldest_Year=9999
        List = [Label, Calc]
        for i in List:
            for Key in i:
                Year = int(Key[2:6])
                if Year<Oldest_Year:
                    Oldest_Year = Year
                    List_to_use = i 

        New_Dictionary = List_to_use

        for i in List:
            if i == List_to_use:
                for Key in i:
                    if i[Key] == {}:
                        for item in List:
                            if item != List_to_use:
                                for UNI_Key in item:
                                    if UNI_Key == Key:
                                        New_Dictionary[Key] = item[UNI_Key]

    return(New_Dictionary)

In [29]:
def MC_fucntion(Shares_Outstanding, price):
    Market_Cap = copy.deepcopy(Shares_Outstanding)
    try:
        for Key in Shares_Outstanding:
            if Shares_Outstanding[Key] != '':
                Market_Cap[Key]['val'] = Shares_Outstanding[Key]['val']*price
    except:
        Market_Cap = {}
    return Market_Cap

In [30]:
def dataForFurtureEarnings(List):
    grabbingData = {
        '10-Q1': None,
        '10-Q2': None,
        '10-Q3': None,
        '10-Q4': None,
        '10-K':None
    }

    KYear = 0
    Q1Year = 0 
    Q2Year = 0 
    Q3Year = 0 
    Q4Year = 0 


    for item in List: 
        try:
            for info in item:
                if item[info] != '':
                    if info[-2:-1] != 'Q':
                        currentDataValue = int(info[2:6])
                        if currentDataValue > KYear:
                            KYear = currentDataValue
                            grabbingData['10-K'] = item[info]['filed']                
            for info in item:
                if item[info] != '':
                    if info[-2:] == 'Q1':
                        currentDataValue = int(info[2:6])
                        if currentDataValue > Q1Year:
                            Q1Year = currentDataValue
                            grabbingData['10-Q1'] = item[info]['filed']             
            for info in item:
                if item[info] != '':
                    if info[-2:] == 'Q2':
                        currentDataValue = int(info[2:6])
                        if currentDataValue > Q2Year:
                            Q2Year = currentDataValue
                            grabbingData['10-Q2'] = item[info]['filed']
            for info in item:
                if item[info] != '':
                    if info[-2:] == 'Q3':
                        currentDataValue = int(info[2:6])
                        if currentDataValue > Q3Year:
                            Q3Year = currentDataValue
                            grabbingData['10-Q3'] = item[info]['filed']
            for info in item:
                if item[info] != '':
                    if info[-2:] == 'Q4':
                        currentDataValue = int(info[2:6])
                        if currentDataValue > Q4Year:
                            Q4Year = currentDataValue
                            grabbingData['10-Q4'] = item[info]['filed']
        except:
            x=1

    # Deleting the quarterly report that matches with the 10-K
    x = None
    for forms in grabbingData:
        if forms != '10-K' and grabbingData[forms] == grabbingData['10-K']:
            x = forms
    if x != None:
        del grabbingData[x]

    # Expected Earning Dates
    for item in grabbingData:
        if grabbingData[item] != None:
            datetime_str = grabbingData[item]
            datetime_object = datetime.strptime(datetime_str, '%Y-%m-%d')
            grabbingData[item] = datetime_object + relativedelta(years=1)

    return(grabbingData)

# dataForFurtureEarnings(List)

In [31]:
def autofillMisingQuarter(yr, qr): 

    # Copy the list
    fiscalYearList = copy.deepcopy(yr)
    quarterlyList = copy.deepcopy(qr)

    fiscalYearQuarters = {}
    for items in fiscalYearList:
        try:
            fiscalYearQuarters[items] = {
                'start': fiscalYearList[items]['start'],
                'end': fiscalYearList[items]['end'],
                'val': fiscalYearList[items]['val'],
                'Q1': {'val': None, 'frame': None},
                'Q2': {'val': None, 'frame': None},
                'Q3': {'val': None, 'frame': None},
                'Q4': {'val': None, 'frame': None},
            }
        except Exception as err:
    #         print(err)
            x=1

    for frames in quarterlyList:
        try:
            for items in fiscalYearQuarters:
                end = datetime.strptime(fiscalYearQuarters[items]['end'], '%Y-%m-%d')
                start = datetime.strptime(fiscalYearQuarters[items]['start'], '%Y-%m-%d')
                framesDate = datetime.strptime(quarterlyList[frames]['end'], '%Y-%m-%d')
                if framesDate>start and framesDate<end:
                    for keys in fiscalYearQuarters[items]:
                        if frames[-2:] == keys: 
                            fiscalYearQuarters[items][keys]['val'] = quarterlyList[frames]['val']
                            fiscalYearQuarters[items][keys]['frame'] = quarterlyList[frames]['frame']

        except Exception as err:
    #         print(err)
            x=1

    for frames in fiscalYearQuarters:
        counter = 0 
        for key in fiscalYearQuarters[frames]:
            try: 
                if fiscalYearQuarters[frames][key]['val'] == None:
                    counter = counter + 1 
                    nullValue = key
            except Exception as err:
    #             print(err)
                x=1

        if counter == 1:
            sumList = ['Q1','Q2','Q3','Q4']
            sumList.remove(nullValue)
            Sum = 0
            for quarters in sumList:
                Sum = Sum + fiscalYearQuarters[frames][quarters]['val']
            missingVal = fiscalYearQuarters[frames]['val'] - Sum
            fiscalYearQuarters[frames][nullValue]['val'] = missingVal
            fiscalYearQuarters[frames][nullValue]['frame'] = 'Calc'

    for frames in quarterlyList:
        if quarterlyList[frames] == '':
            quarter = frames[-2:]
            for items in fiscalYearQuarters:
                for keys in fiscalYearQuarters[items]:
                    try: 
                        if fiscalYearQuarters[items][keys]['frame'] == 'Calc':
                            year = fiscalYearQuarters[items]['end'][:4]
                            frameLabel = "CY{}{}".format(year, keys)
                            if frameLabel == frames:
                                quarterlyList[frames] = copy.deepcopy(fiscalYearList[items])
                                quarterlyList[frames]['val'] = fiscalYearQuarters[items][quarter]['val']
                                quarterlyList[frames]['start'] = None
                                quarterlyList[frames]['label'] = 'Calc'                        
                    except:
                        x=1

    return(yr, quarterlyList)

# Data = autofillMisingQuarter(Revenue[0], Revenue[1])
# Data
# fiscalYearList

In [32]:
def TTM(Quarterly):
    try: 
        Quarter_data = copy.deepcopy(Quarterly)

        # Collected the First Year of data 
        for Key in Quarter_data:
            try:
                #This line triggers an error
                end = Quarter_data[Key]['end']
                FirstYear = Key
                break
            except TypeError:
                x=1
        FirstYear = int(FirstYear[2:6])

        # Collecting the Last Key of Data 
        for Key in Quarter_data:
            try:
                #This line triggers an error
                end = Quarter_data[Key]['end']
                LastKey = Key
            except TypeError:
                x=1

        LastYear = int(LastKey[2:6])
        QuarterID = LastKey[-2:]

        TTM = {}
        TimeDiff = LastYear-FirstYear+1
        for num in range(TimeDiff):
            YearID = FirstYear+num
            TTM_Frame = 'TTM{}{}'.format(YearID, QuarterID)
            TTM[TTM_Frame] = {}

        QuarterNum = int(LastKey[-1])
        for Key in TTM:
            YearKey = int(Key[3:7])
            QuarterNum = int(LastKey[-1])
            for num in range(4):
                KeyNum = QuarterNum-num
                if KeyNum == 0:
                    # Refresh quarter and change year  
                    QuarterNum = 4+num
                    KeyNum = 4
                    YearKey=YearKey-1
                QRFrame = 'CY{}Q{}'.format(YearKey, KeyNum)
                TTM[Key][QRFrame]= None

        for Key in TTM:
            for QRFrame in TTM[Key]:
                try:
                    TTM[Key][QRFrame] = Quarter_data[QRFrame]['val']
                except Exception as err:
                    x=1
        counter = 0 
        for Key in TTM:
            counter = counter +1 
            None_Value_Present = 'No'
            Sum_List = []
            for InnerFrame in TTM[Key]:
                if TTM[Key][InnerFrame]==None:
                    None_Value_Present = 'Yes'
                Sum_List.append(TTM[Key][InnerFrame])

            if None_Value_Present == 'No':
                TTM_Value = sum(Sum_List)
                TTM[Key]={}
                TTM[Key]['val'] = TTM_Value
                QRFrame = 'CY{}'.format(Key[3:])
                TTM[Key]['end'] = Quarter_data[QRFrame]['end']
                TTM[Key]['CIK'] = CIK
                TTM[Key]['Ticker'] = Ticker 
                TTM[Key]['filed'] = Quarter_data[QRFrame]['filed']
                TTM[Key]['fy'] = None
                TTM[Key]['fp'] = None
                TTM[Key]['form'] = None
                TTM[Key]['frame'] = Key
                TTM[Key]['label'] = 'Calc'
                TTM[Key]['accn'] = None
                TTM[Key]['Num_From_Current_Date'] = len(TTM)-counter
            else:
                TTM[Key]={}
    except NameError:
        TTM = {}

    return TTM

In [33]:
# List = copy.deepcopy(Revenue[0])
# Table = 'Revenue'

def YoY_Function(Table, List):
    List = copy.deepcopy(List)
    
    try: 
        #Find the older year
        for Key in List: 
            try:
                # This triggers an error that we can use and a filtering method 
                if List[Key]['val'] != None:
                    OldestYear = Key
                    OldestYear= OldestYear.replace('CY', '')
                    OldestYear= OldestYear.replace('TTM', '')
                    OldestYear = int(OldestYear[:4])
                    break
            except: 
                x=1 

        #Find the most current Year
        for Key in List: 
            try:
                # This triggers an error that we can use and a filtering method 
                if List[Key]['val'] != None:
                    RecentDataYear = Key
                    RecentDataYear= RecentDataYear.replace('CY', '')
                    RecentDataYear= RecentDataYear.replace('TTM', '')
                    RecentDataYear = int(RecentDataYear[:4])
            except: 
                x=1 

        # Error Code if the Current Year isn't correct
        # Note Current_Year is the Master Value at the top of the file 
        if Current_Year-RecentDataYear>1:
            x=1/0

        # Identifing the Header to we can know it this CY or TTM 
        if Key[:2]=='CY':
            Header = 'CY'
        else: 
            Header = 'TTM'

        try:
            # Intentional error code to filter quarterly from fiscal year Quarly will error out 
            Num = int(Key[-2:])
            Footer = ''
        except:
            Footer = Key[-2:]

        PresentYear = RecentDataYear
        Function_Dictonary = {}
        counter = 0 
        while PresentYear!=OldestYear:
            counter += 1
            YearAgo = PresentYear-1
            PresentFrame = '{}{}{}'.format(Header, PresentYear, Footer)
            YearAgoFrame = '{}{}{}'.format(Header, YearAgo, Footer)

            #Year Over Year Math
            try: 
                PV = List[PresentFrame]['val']
                YearAgoValue = List[YearAgoFrame]['val']

                YoY = round(((PV-YearAgoValue)/YearAgoValue)*100,2)
                Function_Dictonary[PresentFrame] = {}       
                Function_Dictonary[PresentFrame]['form'] = '{}YoY'.format(counter)
                Function_Dictonary[PresentFrame]['val'] = YoY
                Function_Dictonary[PresentFrame]['end'] = List[PresentFrame]['end']
                Function_Dictonary[PresentFrame]['filed'] = List[PresentFrame]['filed']
                Function_Dictonary[PresentFrame]['frame'] = List[PresentFrame]['frame']
        #         Function_Dictonary[PresentFrame]['start'] = List[YearAgoValue]['end']
                Function_Dictonary[PresentFrame]['cik'] = CIK
                Function_Dictonary[PresentFrame]['ticker'] = Ticker
                Function_Dictonary[PresentFrame]['TableRef'] = Table

        #         print(YoY, PV, YearAgoValue)
            except:
                Function_Dictonary[PresentFrame] = {}
#                 Function_Dictonary[PresentFrame]['TableRef'] = Table
#                 Function_Dictonary[PresentFrame]['frame'] = '{}YoY'.format(counter)
            PresentYear-= 1
    except ZeroDivisionError:
        Function_Dictonary= {}
    except UnboundLocalError:
        Function_Dictonary= {}
    importing_data_into_CalcTable(Function_Dictonary)
    return Function_Dictonary

In [34]:
def CAGR_Function(Table, List):
    List = copy.deepcopy(List)

    try: 
        #Find the older year
        for Key in List: 
            try:
                # This triggers an error that we can use and a filtering method 
                if List[Key]['val'] != None:
                    OldestYear = Key
                    OldestYear= OldestYear.replace('CY', '')
                    OldestYear= OldestYear.replace('TTM', '')
                    OldestYear = int(OldestYear[:4])
                    break
            except: 
                x=1 

        #Find the most current Year
        for Key in List: 
            try:
                # This triggers an error that we can use and a filtering method 
                if List[Key]['val'] != None:
                    RecentDataYear = Key
                    RecentDataYear= RecentDataYear.replace('CY', '')
                    RecentDataYear= RecentDataYear.replace('TTM', '')
                    RecentDataYear = int(RecentDataYear[:4])
            except: 
                x=1 

        # Error Code if the Current Year isn't correct
        # Note Current_Year is the Master Value at the top of the file 
        if Current_Year-RecentDataYear>1:
            x=1/0

        # Identifing the Header to we can know it this CY or TTM 
        if Key[:2]=='CY':
            Header = 'CY'
        else: 
            Header = 'TTM'

        try:
            # Intentional error code to filter quarterly from fiscal year Quarly will error out 
            Num = int(Key[-2:])
            Footer = ''
        except:
            Footer = Key[-2:]

        PresentYear = RecentDataYear
        Function_Dictonary = {}
        counter = 0 
        PresentFrame = '{}{}{}'.format(Header, PresentYear, Footer)
        error_factor = 0 
        while PresentYear!=OldestYear:
            counter += 1
            YearAgo = PresentYear-1
            YearAgoFrame = '{}{}{}'.format(Header, YearAgo, Footer)

            #Year Over Year Math
            try: 
                PV = List[PresentFrame]['val']            
                PV_end_date = datetime.strptime(List[PresentFrame]['end'], '%Y-%m-%d') 
                YearAgoValue = List[YearAgoFrame]['val']
                YearAgoValue_EndDate = datetime.strptime(List[YearAgoFrame]['end'], '%Y-%m-%d') 
                delta = relativedelta(PV_end_date, YearAgoValue_EndDate)
                Years_between = int(delta.years) + int(delta.months)/12 + error_factor
                
                if Years_between<= 0:
                    error_factor = 1
                    Years_between = int(delta.years) + int(delta.months)/12 + error_factor
                
                if YearAgoValue <= 0 or PV <= 0: 
                    x=1/0

                #numpy_financial.rate(nper, pmt, pv, fv)
                CAGR = round(numpy_financial.rate(Years_between, 0, -YearAgoValue, PV)*100, 2)

                Function_Dictonary[YearAgoFrame] = {}       
                Function_Dictonary[YearAgoFrame]['form'] = '{}CAGR'.format(counter)
                Function_Dictonary[YearAgoFrame]['val'] = CAGR
                Function_Dictonary[YearAgoFrame]['end'] = List[YearAgoFrame]['end']
                Function_Dictonary[YearAgoFrame]['filed'] = List[YearAgoFrame]['filed']
                Function_Dictonary[YearAgoFrame]['frame'] = List[YearAgoFrame]['frame']
        #         Function_Dictonary[PresentFrame]['start'] = List[YearAgoValue]['end']
                Function_Dictonary[YearAgoFrame]['cik'] = CIK
                Function_Dictonary[YearAgoFrame]['ticker'] = Ticker
                Function_Dictonary[YearAgoFrame]['TableRef'] = Table

        #         print(YoY, PV, YearAgoValue)
            except:
                Function_Dictonary[YearAgoFrame] = {}
    #                 Function_Dictonary[PresentFrame]['TableRef'] = Table
    #                 Function_Dictonary[PresentFrame]['frame'] = '{}YoY'.format(counter)
            PresentYear-= 1
    except ZeroDivisionError:
        Function_Dictonary= {}
    except UnboundLocalError:
        Function_Dictonary= {}
#     print(Function_Dictonary)
    importing_data_into_CalcTable(Function_Dictonary)
    return Function_Dictonary

In [35]:
# List = copy.deepcopy(Revenue[0])
# Table = 'Revenue'

def YoYAvg_Function(Table, List):
    List = copy.deepcopy(List)
    
    try: 
        #Find the older year
        for Key in List: 
            try:
                # This triggers an error that we can use and a filtering method 
                if List[Key]['val'] != None:
                    OldestYear = Key
                    OldestYear= OldestYear.replace('CY', '')
                    OldestYear= OldestYear.replace('TTM', '')
                    OldestYear = int(OldestYear[:4])
                    break
            except: 
                x=1 

        #Find the most current Year
        for Key in List: 
            try:
                # This triggers an error that we can use and a filtering method 
                if List[Key]['val'] != None:
                    RecentDataYear = Key
                    RecentDataYear= RecentDataYear.replace('CY', '')
                    RecentDataYear= RecentDataYear.replace('TTM', '')
                    RecentDataYear = int(RecentDataYear[:4])
            except: 
                x=1 

        # Error Code if the Current Year isn't correct
        # Note Current_Year is the Master Value at the top of the file 
        if Current_Year-RecentDataYear>1:
            x=1/0

        # Identifing the Header to we can know it this CY or TTM 
        if Key[:2]=='CY':
            Header = 'CY'
        else: 
            Header = 'TTM'

        try:
            # Intentional error code to filter quarterly from fiscal year Quarly will error out 
            Num = int(Key[-2:])
            Footer = ''
        except:
            Footer = Key[-2:]

        PresentYear = RecentDataYear
        Function_Dictonary = {}
        counter = 0 
        YoY_List = []
        while PresentYear!=OldestYear:
            counter += 1
            YearAgo = PresentYear-1
            PresentFrame = '{}{}{}'.format(Header, PresentYear, Footer)
            YearAgoFrame = '{}{}{}'.format(Header, YearAgo, Footer)

            #Year Over Year Math
            try: 
                PV = List[PresentFrame]['val']
                YearAgoValue = List[YearAgoFrame]['val']
                YoY = round(((PV-YearAgoValue)/YearAgoValue)*100,2)
                YoY_List.append(YoY)
                YoYAvg = round(sum(YoY_List)/len(YoY_List),2)
                
                Function_Dictonary[PresentFrame] = {}       
                Function_Dictonary[PresentFrame]['form'] = '{}YoYAvg'.format(counter)
                Function_Dictonary[PresentFrame]['val'] = YoYAvg
                Function_Dictonary[PresentFrame]['end'] = List[PresentFrame]['end']
                Function_Dictonary[PresentFrame]['filed'] = List[PresentFrame]['filed']
                Function_Dictonary[PresentFrame]['frame'] = List[PresentFrame]['frame']
                Function_Dictonary[PresentFrame]['cik'] = CIK
                Function_Dictonary[PresentFrame]['ticker'] = Ticker
                Function_Dictonary[PresentFrame]['TableRef'] = Table
            except:
                Function_Dictonary[PresentFrame] = {}
            PresentYear-= 1
    except ZeroDivisionError:
        Function_Dictonary= {}
    except UnboundLocalError:
        Function_Dictonary= {}
    importing_data_into_CalcTable(Function_Dictonary)
    return Function_Dictonary

In [36]:
# List = copy.deepcopy(Revenue[0])
# Table = 'Revenue'

def Avg_Function(Table, List):
    List = copy.deepcopy(List)
    
    try: 
        #Find the older year
        for Key in List: 
            try:
                # This triggers an error that we can use and a filtering method 
                if List[Key]['val'] != None:
                    OldestYear = Key
                    OldestYear= OldestYear.replace('CY', '')
                    OldestYear= OldestYear.replace('TTM', '')
                    OldestYear = int(OldestYear[:4])
                    break
            except: 
                x=1 

        #Find the most current Year
        for Key in List: 
            try:
                # This triggers an error that we can use and a filtering method 
                if List[Key]['val'] != None:
                    RecentDataYear = Key
                    RecentDataYear= RecentDataYear.replace('CY', '')
                    RecentDataYear= RecentDataYear.replace('TTM', '')
                    RecentDataYear = int(RecentDataYear[:4])
            except: 
                x=1 

        # Error Code if the Current Year isn't correct
        # Note Current_Year is the Master Value at the top of the file 
        if Current_Year-RecentDataYear>1:
            x=1/0

        # Identifing the Header to we can know it this CY or TTM 
        if Key[:2]=='CY':
            Header = 'CY'
        else: 
            Header = 'TTM'

        try:
            # Intentional error code to filter quarterly from fiscal year Quarly will error out 
            Num = int(Key[-2:])
            Footer = ''
        except:
            Footer = Key[-2:]

        PresentYear = RecentDataYear
        Function_Dictonary = {}
        counter = 0 
        PV_List = []
        while PresentYear!=OldestYear:
            counter += 1
            YearAgo = PresentYear-1
            PresentFrame = '{}{}{}'.format(Header, PresentYear, Footer)
            YearAgoFrame = '{}{}{}'.format(Header, YearAgo, Footer)

            #Year Over Year Math
            try: 
                PV = List[PresentFrame]['val']
                PV_List.append(PV)
                Avg = round(sum(PV_List)/len(PV_List),2)
                
                Function_Dictonary[PresentFrame] = {}       
                Function_Dictonary[PresentFrame]['form'] = '{}Avg'.format(counter)
                Function_Dictonary[PresentFrame]['val'] = Avg
                Function_Dictonary[PresentFrame]['end'] = List[PresentFrame]['end']
                Function_Dictonary[PresentFrame]['filed'] = List[PresentFrame]['filed']
                Function_Dictonary[PresentFrame]['frame'] = List[PresentFrame]['frame']
                Function_Dictonary[PresentFrame]['cik'] = CIK
                Function_Dictonary[PresentFrame]['ticker'] = Ticker
                Function_Dictonary[PresentFrame]['TableRef'] = Table
            except:
                Function_Dictonary[PresentFrame] = {}
            PresentYear-= 1
    except ZeroDivisionError:
        Function_Dictonary= {}
    except UnboundLocalError:
        Function_Dictonary= {}
    importing_data_into_CalcTable(Function_Dictonary)
    return Function_Dictonary

In [37]:
def Combination_of_Calc_Func(Table, Dataset, x_list):
    if 'YoY' in x_list:
        YoY_Function(Table, Dataset)
    if 'CAGR' in x_list:
        CAGR_Function(Table, Dataset) 
    if 'YoYAvg' in x_list:
        YoYAvg_Function(Table, Dataset)
    if 'Avg' in x_list:
        Avg_Function(Table, Dataset)

In [38]:
cursor.close()
cnx.close()